# Outlier Validation App (App 2)

This interactive visualization tool was used to present the **validation of our outlier detection analysis** in the final presentation of the Constructor Learning Bootcamp.

This is the structure of the notebook:

**1. Import modules**
    
* Import all modules needed to run the notebook. Install as needed.

**2. Load data and sample**

* The data that are loaded are preprepared in the notebook **"presentation.iynb"** and imported below as a csv file **"tsne_metrics.csv"**
* Prepare the data for the app, i.e., rename variables and reshape data
* Select a sample and subset of the data for presenting the app, skip if you want to work with all data
* Normalize variables

**3. Create outlier validation app**

* Design (e.g., app size, labels, font sizes) should be changed for internal use, they were adjusted for the final presentation
* If one aims to run the app on 180,000k motion sequences 
    * Either: Design should be changed
    * Or: Code from Load data and sample should be added to the app as new functionality

# 1. Import modules

In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

import dash
from dash import dcc, html, Input, Output

# 2. Load data and sample

In [3]:
df_tsne_metrics = pd.read_csv("../data/processed/tsne_metrics.csv", index_col=[0])

# rename variables for better labels in app
df_tsne_metrics.rename(columns={'0': 'Projection 1'}, inplace=True)
df_tsne_metrics.rename(columns={'1': 'Projection 2'}, inplace=True)
df_tsne_metrics.rename(columns={'action': 'Action'}, inplace=True)

df_tsne_metrics

,Projection 1,Projection 2,idx,Action,mean_mpjpe_STSGCN,mean_mpjpe_motionmixer,CBLOF_full,CBLOF_Ti_10,CBLOF_To_25,IFOREST_full,IFOREST_Ti_10,IFOREST_To_25,HDBSCAN_full,HDBSCAN_Ti_10,HDBSCAN_To_25,LOF_full,LOF_Ti_10,LOF_To_25,min_mean_mpjpe
0,-1.491875,-108.968216,0,walking,93.675567,102.448229,6.826516,4.801257,6.284274,-0.071506,-0.103554,-0.061220,0.225378,0.129219,0.096772,1.342149,1.133492,1.298507,93.675567
1,-1.542240,-108.997650,1,walking,85.310110,94.610764,6.778658,4.784079,6.272123,-0.070384,-0.103796,-0.057540,0.217411,0.125704,0.097167,1.328314,1.136863,1.291393,85.310110
2,-1.600920,-109.033640,2,walking,83.354780,99.023203,6.705774,4.822663,6.241388,-0.074504,-0.103825,-0.056933,0.205538,0.134777,0.093353,1.309284,1.149762,1.280669,83.354780
3,-1.678499,-109.072540,3,walking,87.318396,104.901132,6.608236,4.907010,6.178011,-0.071427,-0.100295,-0.057331,0.189529,0.150669,0.082388,1.284328,1.167995,1.262712,87.318396
4,-1.787714,-109.122210,4,walking,89.802102,107.749316,6.323247,4.981930,6.099157,-0.072563,-0.097557,-0.062177,0.173460,0.165176,0.070758,1.262906,1.181005,1.244329,89.802102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180072,-67.498130,-76.802600,9270,walkingtogether,37.963082,39.220811,4.280818,5.447292,4.396982,-0.077916,-0.087516,-0.092054,0.346219,0.025580,0.023360,1.041184,1.073231,1.065025,37.963082
180073,-67.102540,-76.509630,9271,walkingtogether,37.647556,34.372172,4.432358,5.610270,4.265305,-0.079021,-0.081029,-0.095020,0.345278,0.038833,0.004577,1.039324,1.074453,1.061717,34.372172
180074,-66.749420,-76.127110,9272,walkingtogether,41.737064,32.446786,4.631757,5.733671,4.174752,-0.079837,-0.083097,-0.099845,0.341166,0.044855,0.083452,1.038706,1.072405,1.059400,32.446786
180075,-66.459190,-75.689150,9273,walkingtogether,39.341552,31.302170,4.869488,5.748328,4.188549,-0.082146,-0.084715,-0.097257,0.336184,0.036809,0.058962,1.038026,1.070272,1.054587,31.302170


In [4]:
# function to sample randomly from actions using a list actions

def sample_by_action(df, n = 1000, actions=['walking', 'eating'], random_state = 42):
    df_sample = pd.DataFrame()
    for action in actions:
        df_sample= pd.concat([df_sample, df[df.Action == action].sample(n=n, random_state=random_state)])
    return df_sample

In [5]:
df_tsne_metrics["Action"].unique()

array(['walking', 'eating', 'smoking', 'discussion', 'directions',
       'greeting', 'phoning', 'posing', 'purchases', 'sitting',
       'sittingdown', 'takingphoto', 'waiting', 'walkingdog',
       'walkingtogether'], dtype=object)

In [6]:
# use all actions here, because this app uses actions in a dropdown
actions = ['walking', 'eating', 'smoking', 'discussion', 'directions',
       'greeting', 'phoning', 'posing', 'purchases', 'sitting',
       'sittingdown', 'takingphoto', 'waiting', 'walkingdog',
       'walkingtogether']
df_sample = sample_by_action(df_tsne_metrics, n=100, actions=actions, random_state = 42)
df_sample

,Projection 1,Projection 2,idx,Action,mean_mpjpe_STSGCN,mean_mpjpe_motionmixer,CBLOF_full,CBLOF_Ti_10,CBLOF_To_25,IFOREST_full,IFOREST_Ti_10,IFOREST_To_25,HDBSCAN_full,HDBSCAN_Ti_10,HDBSCAN_To_25,LOF_full,LOF_Ti_10,LOF_To_25,min_mean_mpjpe
3116,-93.167620,74.029290,3116,walking,37.308055,35.490965,4.849226,6.953999,5.722817,-0.073104,-0.055284,-0.065945,0.010343,0.013739,0.014408,1.041582,1.172646,1.100537,35.490965
14490,-71.934350,-61.324726,14490,walking,44.560363,51.674447,5.315620,6.029859,5.764158,-0.079457,-0.051268,-0.074286,0.124446,0.076333,0.027187,1.123687,1.103043,1.085511,44.560363
14416,-73.751980,-63.028355,14416,walking,45.434795,52.276969,5.197551,5.734965,5.694318,-0.093236,-0.085196,-0.074192,0.038283,0.042022,0.005718,1.069552,1.113829,1.085680,45.434795
14711,-51.916620,-69.005280,14711,walking,32.930991,43.683820,5.149294,5.658388,5.455088,-0.058082,-0.078576,-0.083818,0.069968,0.059675,0.048870,1.077815,1.054475,1.049560,32.930991
3307,-0.490633,-89.492744,3307,walking,49.074074,40.456684,7.142880,5.659097,7.190692,-0.035939,-0.075651,-0.046680,0.105932,0.018158,0.075175,1.065702,1.105696,1.101048,40.456684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172567,-76.683720,-80.763885,1765,walkingtogether,106.678383,116.575575,8.511077,7.156005,8.388285,-0.043406,-0.066705,-0.021579,0.193097,0.045247,0.135146,1.551568,1.453742,1.554393,106.678383
174111,-96.155426,84.970535,3309,walkingtogether,94.013496,93.256885,8.758814,9.157292,6.905630,0.015341,-0.029247,-0.041484,0.205514,0.070489,0.009460,1.443798,1.340144,1.279081,93.256885
177103,-74.319786,-97.769670,6301,walkingtogether,31.063853,33.501992,7.175620,5.958531,5.982388,-0.060987,-0.075502,-0.063266,0.059997,0.041430,0.013888,1.114006,1.098095,1.082603,31.063853
179874,-71.640050,-75.945450,9072,walkingtogether,42.525311,39.453849,5.290853,4.414806,5.009238,-0.095465,-0.080828,-0.086174,0.294772,0.019099,0.032935,1.014683,1.038581,1.024317,39.453849


In [7]:
# reshape data to get a column for the number of frames (needed! for this app)

suffixes = ['_full', '_Ti_10', '_To_25']

dfs_reshaped = []

for suffix in suffixes:
    cols_current = [col for col in df_sample.columns if col.endswith(suffix)]
    other_cols = [col for col in df_sample.columns if not any(col.endswith(s) for s in suffixes)]

    df_current = df_sample[other_cols + cols_current]
    df_current["Number of frames"] = suffix[1:]
    df_current.columns = df_current.columns.str.replace(suffix, '') 
    dfs_reshaped.append(df_current)

df_sample_reshaped = pd.concat(dfs_reshaped)
df_sample_reshaped 

C:\Users\Jonas\AppData\Local\Temp\ipykernel_23388\2744349630.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_current["Number of frames"] = suffix[1:]
C:\Users\Jonas\AppData\Local\Temp\ipykernel_23388\2744349630.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_current["Number of frames"] = suffix[1:]
C:\Users\Jonas\AppData\Local\Temp\ipykernel_23388\2744349630.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

,Projection 1,Projection 2,idx,Action,mean_mpjpe_STSGCN,mean_mpjpe_motionmixer,min_mean_mpjpe,CBLOF,IFOREST,HDBSCAN,LOF,Number of frames
3116,-93.167620,74.029290,3116,walking,37.308055,35.490965,35.490965,4.849226,-0.073104,0.010343,1.041582,full
14490,-71.934350,-61.324726,14490,walking,44.560363,51.674447,44.560363,5.315620,-0.079457,0.124446,1.123687,full
14416,-73.751980,-63.028355,14416,walking,45.434795,52.276969,45.434795,5.197551,-0.093236,0.038283,1.069552,full
14711,-51.916620,-69.005280,14711,walking,32.930991,43.683820,32.930991,5.149294,-0.058082,0.069968,1.077815,full
3307,-0.490633,-89.492744,3307,walking,49.074074,40.456684,40.456684,7.142880,-0.035939,0.105932,1.065702,full
...,...,...,...,...,...,...,...,...,...,...,...,...
172567,-76.683720,-80.763885,1765,walkingtogether,106.678383,116.575575,106.678383,8.388285,-0.021579,0.135146,1.554393,To_25
174111,-96.155426,84.970535,3309,walkingtogether,94.013496,93.256885,93.256885,6.905630,-0.041484,0.009460,1.279081,To_25
177103,-74.319786,-97.769670,6301,walkingtogether,31.063853,33.501992,31.063853,5.982388,-0.063266,0.013888,1.082603,To_25
179874,-71.640050,-75.945450,9072,walkingtogether,42.525311,39.453849,39.453849,5.009238,-0.086174,0.032935,1.024317,To_25


In [8]:
# use all possible number of frames, no subsetting needed
df = df_sample_reshaped

In [9]:
# normalize such that all relevant variables are in range 0-1 

columns_to_normalize = ['mean_mpjpe_STSGCN', 'mean_mpjpe_motionmixer', 'min_mean_mpjpe', 'CBLOF', 'IFOREST', 'HDBSCAN', 'LOF']

scaler = MinMaxScaler()

df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])
df

,Projection 1,Projection 2,idx,Action,mean_mpjpe_STSGCN,mean_mpjpe_motionmixer,min_mean_mpjpe,CBLOF,IFOREST,HDBSCAN,LOF,Number of frames
3116,-93.167620,74.029290,3116,walking,0.105138,0.114846,0.114846,0.152969,0.207793,0.018724,0.057812,full
14490,-71.934350,-61.324726,14490,walking,0.129677,0.172211,0.146994,0.187781,0.181955,0.225272,0.117174,full
14416,-73.751980,-63.028355,14416,walking,0.132636,0.174347,0.150094,0.178968,0.125911,0.069300,0.078034,full
14711,-51.916620,-69.005280,14711,walking,0.090327,0.143887,0.105772,0.175366,0.268890,0.126656,0.084008,full
3307,-0.490633,-89.492744,3307,walking,0.144950,0.132448,0.132448,0.324166,0.358951,0.191758,0.075250,full
...,...,...,...,...,...,...,...,...,...,...,...,...
172567,-76.683720,-80.763885,1765,walkingtogether,0.339865,0.402263,0.367181,0.417123,0.417357,0.244642,0.428580,To_25
174111,-96.155426,84.970535,3309,walkingtogether,0.297011,0.319606,0.319606,0.306458,0.336399,0.017124,0.229526,To_25
177103,-74.319786,-97.769670,6301,walkingtogether,0.084009,0.107796,0.099153,0.237548,0.247805,0.025140,0.087470,To_25
179874,-71.640050,-75.945450,9072,walkingtogether,0.122791,0.128893,0.128893,0.164913,0.154635,0.059619,0.045329,To_25


# 3. Create outlier validation app

In [61]:
# initialize dash app
app = dash.Dash(__name__)

# define layout of the app
app.layout = html.Div([
    html.Div([
        # Select Your Analysis heading
        html.H4("Select Your Analysis", style={'font-size': '24px'}),

        # dropdown for x-axis decision score
        html.Label("Select Outlier Detection Model:", style={'font-size': '20px'}),
        dcc.Dropdown(
            id='x-decision-score',
            options=[
                {'label': 'CBLOF', 'value': 'CBLOF'},
                {'label': 'HDBSCAN', 'value': 'HDBSCAN'},
                {'label': 'IFOREST', 'value': 'IFOREST'},
                {'label': 'LOF', 'value': 'LOF'}
            ],
            value='CBLOF',  # default value
            style={'width': '400px', 'margin-bottom':'10px'}
        ),

        # dropdown for y-axis prediction error
        html.Label("Select Motion Model:", style={'font-size': '20px'}),
        dcc.Dropdown(
            id='y-prediction-error',
            options=[
                {'label': 'STSGCN', 'value': 'mean_mpjpe_STSGCN'},
                {'label': 'motionmixer (MM)', 'value': 'mean_mpjpe_motionmixer'},
                {'label': 'min(STSGCN, MM)', 'value': 'min_mean_mpjpe'}
            ],
            value='mean_mpjpe_STSGCN',  # default value
            style={'width': '400px', 'margin-bottom':'10px'}
        ),

        # dropdown for filtering by action
        html.Label("Filter by Action:", style={'font-size': '20px'}),
        dcc.Dropdown(
            id='action-filter',
            options=[
                {'label': 'walking', 'value': 'walking'},
                {'label': 'eating', 'value': 'eating'},
                {'label': 'smoking', 'value': 'smoking'},
                {'label': 'discussion', 'value': 'discussion'},
                {'label': 'directions', 'value': 'directions'},
                {'label': 'greeting', 'value': 'greeting'},
                {'label': 'phoning', 'value': 'phoning'},
                {'label': 'posing', 'value': 'posing'},
                {'label': 'purchases', 'value': 'purchases'},
                {'label': 'sitting', 'value': 'sitting'},
                {'label': 'sittingdown', 'value': 'sittingdown'},
                {'label': 'takingphoto', 'value': 'takingphoto'},
                {'label': 'waiting', 'value': 'waiting'},
                {'label': 'walkingdog', 'value': 'walkingdog'},
                {'label': 'walkingtogether', 'value': 'walkingtogether'},
            ],
            value=None,  # default value (None means no filter is applied)
            style={'width': '400px', 'margin-bottom':'10px'}
        ),

        # dropdown for filtering by n_frames
        html.Label("Filter by Number of frames:", style={'font-size': '20px'}),
        dcc.Dropdown(
            id='n-frames-filter',
            options=[
                {'label': 'Input Only (First 10)', 'value': 'Ti_10'},
                {'label': 'Output Only (Last 25)', 'value': 'To_25'},
                {'label': 'Full (35)', 'value': 'full'},
            ],
            value='full',
            style={'width': '400px', 'margin-bottom':'20px'}
        ),

        # sliders
        html.Label("Set Decision Score Threshold", style={'font-size': '20px'}),
        dcc.Slider(
            id='x-threshold',
            min=0.7,
            max=0.99,
            step=0.01,
            value=0.90,
            marks={i / 100: f"{i / 100:.2f}" for i in range(70, 100, 5)}
        ),
        html.Label("Set Prediction Error Threshold", style={'font-size': '20px'}),
        dcc.Slider(
            id='y-threshold',
            min=0.7,
            max=0.99,
            step=0.01,
            value=0.85,
            marks={i / 100: f"{i / 100:.2f}" for i in range(70, 100, 5)}
        )
    ], style={'width': '400px', 'float': 'left', 'margin-top': '60px', 'margin-left': '20px', 'margin-right': '20px', 'background-color': 'white', 'font-family': 'Arial'}),

    html.Div([
        # scatter plot
        dcc.Graph(id='scatter-plot', style={'width': '1000px', 'height': '600px'})
    ], style={'float': 'left'}),
], style={'font-family': 'Arial'})

# callback to update the scatter plot based on threshold values and selected dropdown values
@app.callback(
    Output('scatter-plot', 'figure'),
    [Input('x-decision-score', 'value'),
     Input('y-prediction-error', 'value'),
     Input('x-threshold', 'value'),
     Input('y-threshold', 'value'),
     Input('action-filter', 'value'),
     Input('n-frames-filter', 'value')]
)

def update_scatter_plot(x_decision_score, y_prediction_error, x_threshold, y_threshold, action_filter, n_frames_filter):

    filtered_df = df.copy()

    # apply filters based on action and n_frames
    if action_filter is not None:
        filtered_df = filtered_df[filtered_df['Action'] == action_filter]

    if n_frames_filter != 'all (35)':
        filtered_df = filtered_df[filtered_df['Number of frames'] == n_frames_filter]

    # filter df by selected decision score and prediction error columns
    filtered_df = filtered_df[[x_decision_score, y_prediction_error, 'idx', 'Action']]

    x_quantile = np.percentile(filtered_df[x_decision_score], x_threshold * 100)
    y_quantile = np.percentile(filtered_df[y_prediction_error], y_threshold * 100)

    # group the data based on the thresholds
    true_positive = (filtered_df[x_decision_score] > x_quantile) & (filtered_df[y_prediction_error] > y_quantile)
    true_negative = (filtered_df[x_decision_score] <= x_quantile) & (filtered_df[y_prediction_error] <= y_quantile)
    false_negative = (filtered_df[x_decision_score] <= x_quantile) & (filtered_df[y_prediction_error] > y_quantile)
    false_positive = (filtered_df[x_decision_score] > x_quantile) & (filtered_df[y_prediction_error] <= y_quantile)

    # calculate the counts for each group and precision and recall
    true_positives_count = true_positive.sum()
    true_negatives_count = true_negative.sum()
    false_negatives_count = false_negative.sum()
    false_positives_count = false_positive.sum()

    precision = round(true_positives_count / (true_positives_count + false_positives_count), 2)
    recall = round(true_positives_count / (true_positives_count + false_negatives_count), 2)

    precision_formatted = format(precision, ".2f") 
    recall_formatted = format(recall, ".2f")

    # create separate traces for each group
    traces = [
        {
            'x': filtered_df[x_decision_score][false_positive],
            'y': filtered_df[y_prediction_error][false_positive],
            'mode': 'markers',
            'marker': {'symbol':'square', 'color': '#ca7cd8', 'opacity': 0.6, 'size':'10', 'line':{'color':'black', 'width':'1'}},
            'name': 'False positive                              ',
            'hoverinfo': 'text',  # set hover information to display custom text
            'text': filtered_df[false_positive].apply(lambda row: f"idx: {row['idx']}<br>"
                                                                 f"Action: {row['Action']}<br>"
                                                                 f"{x_decision_score}: {row[x_decision_score]:.2f}<br>"
                                                                 f"{y_prediction_error}: {row[y_prediction_error]:.2f}",
                                                      axis=1).tolist()
        },
        {
            'x': filtered_df[x_decision_score][false_negative],
            'y': filtered_df[y_prediction_error][false_negative],
            'mode': 'markers',
            'marker': {'symbol':'diamond', 'color': 'orange', 'opacity': 0.6, 'size':'10',  'line':{'color':'black', 'width':'1'}},
            'name': 'False negative',
            'hoverinfo': 'text',  # set hover information to display custom text
            'text': filtered_df[false_negative].apply(lambda row: f"idx: {row['idx']}<br>"
                                                                 f"Action: {row['Action']}<br>"
                                                                 f"{x_decision_score}: {row[x_decision_score]:.2f}<br>"
                                                                 f"{y_prediction_error}: {row[y_prediction_error]:.2f}",
                                                      axis=1).tolist()
        },
        {
            'x': filtered_df[x_decision_score][true_positive],
            'y': filtered_df[y_prediction_error][true_positive],
            'mode': 'markers',
            'marker': {'symbol':'triangle-up', 'color': '#7BBA4A', 'opacity': 0.95, 'size':'14',  'line':{'color':'black', 'width':'1'}},
            'name': 'True positive',
            'hoverinfo': 'text',  # set hover information to display custom text
            'text': filtered_df[true_positive].apply(lambda row: f"idx: {row['idx']}<br>"
                                                                f"Action: {row['Action']}<br>"
                                                                f"{x_decision_score}: {row[x_decision_score]:.2f}<br>"
                                                                f"{y_prediction_error}: {row[y_prediction_error]:.2f}",
                                                     axis=1).tolist()
        },
        {
            'x': filtered_df[x_decision_score][true_negative],
            'y': filtered_df[y_prediction_error][true_negative],
            'mode': 'markers',
            'marker': {'symbol':'circle', 'color': '#535354', 'opacity': 0.4, 'size':'8', 'line':{'color':'black', 'width':'1'}},
            'name': 'True negative',
            'hoverinfo': 'text',  # set hover information to display custom text
            'text': filtered_df[true_negative].apply(lambda row: f"idx: {row['idx']}<br>"
                                                                f"Action: {row['Action']}<br>"
                                                                f"{x_decision_score}: {row[x_decision_score]:.2f}<br>"
                                                                f"{y_prediction_error}: {row[y_prediction_error]:.2f}",
                                                     axis=1).tolist()
        }
    ]

    # create the scatter plot figure
    figure = {
        'data': traces,
        'layout': {
            'title': {'text':'<b>Outlier Validation App</b>',
               'font': {'size': 24, 'color':'black', 'family': 'Arial'}},
            'xaxis': {'title': 'Decision Score', 'title_font': {'size': 24}},
            'yaxis': {'title': 'Prediction Error', 'title_font': {'size': 24}},
            'shapes': [
                # add vertical line for x threshold
                {
                    'type': 'line',
                    'x0': x_quantile,
                    'x1': x_quantile,
                    'y0': 0,
                    'y1': 1,
                    'line': {'color': 'gray', 'width': 2, 'dash': 'dash'}
                },
                # add horizontal line for x threshold
                {
                    'type': 'line',
                    'x0': 0,
                    'x1': 1,
                    'y0': y_quantile,
                    'y1': y_quantile,
                    'line': {'color': 'gray', 'width': 2, 'dash': 'dash'}
                }
            ],
            'legend': {'x': 1, 'y': 1, 'font': {'size': 18}},  # position the legend
                'annotations': [
                # annotation for Counts
                {
                    'x': 1.21,
                    'y': 0.5,
                    'xref': 'paper',
                    'yref': 'paper',
                    'text': f'<b>Counts</b>',
                    'showarrow': False,
                    'font': {'size': 18, 'color': 'black'},
                },
                # annotation for Metrics
                {
                    'x': 1.42,
                    'y': 0.5,
                    'xref': 'paper',
                    'yref': 'paper',
                    'text': f'<b>Metrics</b>',
                    'showarrow': False,
                    'font': {'size': 18, 'color': 'black'},
                },
                # annotation for true positives
                {
                    'x': 1.21,
                    'y': 0.4,
                    'xref': 'paper',
                    'yref': 'paper',
                    'text': f'TP: {true_positives_count}',
                    'showarrow': False,
                    'font': {'size': 18, 'color': '#7BBA4A'},
                },
                # annotation for true negatives
                {
                    'x': 1.25,
                    'y': 0.3,
                    'xref': 'paper',
                    'yref': 'paper',
                    'text': f'TN: {true_negatives_count}',
                    'showarrow': False,
                    'font': {'size': 18, 'color': '#535354'},
                },
                # annotation for false negatives
                {
                    'x': 1.23,
                    'y': 0.23,
                    'xref': 'paper',
                    'yref': 'paper',
                    'text': f'FN: {false_negatives_count}',
                    'showarrow': False,
                    'font': {'size': 18, 'color': 'orange'},
                },
                # annotation for false positives
                {
                    'x': 1.21,
                    'y': 0.16,
                    'xref': 'paper',
                    'yref': 'paper',
                    'text': f'FP: {false_positives_count}',
                    'showarrow': False,
                    'font': {'size': 18, 'color': '#ca7cd8'},
                },
                # annotation for precision
                {
                    'x': 1.53,
                    'y': 0.4,
                    'xref': 'paper',
                    'yref': 'paper',
                    'text': f'Precision: {precision_formatted}',
                    'showarrow': False,
                    'font': {'size': 18, 'color': 'black'},
                },
                # annotation for recall
                {
                    'x': 1.48,
                    'y': 0.3,
                    'xref': 'paper',
                    'yref': 'paper',
                    'text': f'Recall: {recall_formatted}',
                    'showarrow': False,
                    'font': {'size': 18, 'color': 'black'},
                },
            ],
        },
    }

    return figure

# run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8052)

In [ ]:
http://127.0.0.1:8052